## Project 4 - Notebook 1 - WebScrapping (by Wilson Alberto Torres)

In [24]:
# importing libraries
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import urllib.request, urllib.parse, urllib.error
from time import sleep

In [2]:
# Instantiation of the web-driver: Selenium
chrome_options = webdriver.ChromeOptions()
chrome_options.headless = True
driver_1 = webdriver.Chrome(executable_path="./chromedriver/chromedriver",chrome_options=chrome_options)
url_root = "https://www.seek.com.au/"
#driver.get(url_root)

<ipython-input-2-ed93348a8973>:3: DeprecationWarning: use options instead of chrome_options
  driver_1 = webdriver.Chrome(executable_path="./chromedriver/chromedriver",chrome_options=chrome_options)


In [3]:
# This function generates a list of the URLs for web-scraping
def list_url_def(url_base,job,location,max,sal_range):
    ret_list = []
    for i in range(1,max+1):
        url_str = url_base+job+'-jobs/in-'+location+'?page='+str(i)+'&salaryrange='+sal_range+'&salarytype=annual'
        #print(url_str)
        ret_list.append(url_str)
    return ret_list

In [4]:
# This function scrapes job information from the website "www.seek.com.au"
def web_scrape_func(list_urls, s_keyword, loc, range_sal):
    data_jobs = []

    for each_url in list_urls:
        driver = webdriver.Chrome(executable_path="./chromedriver/chromedriver",chrome_options=chrome_options)
        print(each_url)
        driver.get(each_url)

        for eachp in range(20):
            each_job = {}
            each_job['s_keyword'] = s_keyword
            each_job['s_loc'] = loc
            each_job['range_sal'] = range_sal
            each_job['job_title'] = driver.find_elements_by_xpath('//article')[eachp].get_attribute('aria-label')
            each_job['job_id'] = driver.find_elements_by_xpath('//article')[eachp].get_attribute('data-job-id')
            id_str = each_job['job_id']
            #print(id_str)

            try:
                str_city = '//article[@data-job-id="'+id_str+'"]/div[1]/span[2]/span/strong/span/a'
                each_job['city'] = driver.find_element_by_xpath(str_city).text
            except:
                each_job['city'] = 'no info'

            str_aurl = '//article[@data-job-id="'+id_str+'"]/span[5]/span/a'
            try:
                each_job['job_agency_url'] = driver.find_element_by_xpath(str_aurl).get_attribute('href')
            except:
                each_job['job_agency_url'] = 'no info'

            try:
                each_job['job_agency'] = driver.find_element_by_xpath(str_aurl).text
            except:
                each_job['job_agency'] = 'no info'

            try:
                str_type = '//article[@data-job-id="'+id_str+'"]/div[1]/span[1]/p'
                each_job['job_type'] = driver.find_element_by_xpath(str_type).text
            except:
                each_job['job_type'] = 'no info'

            try:
                str_sal = '//article[@data-job-id="'+id_str+'"]/div[1]/span[@data-automation="jobSalary"]'
                each_job['job_salary'] = driver.find_element_by_xpath(str_sal).text
            except:
                each_job['job_salary'] = 'no info'

            try:
                str_fld = '//article[@data-job-id="'+id_str+'"]/div[1]/span[4]/span/span/strong/span/a'
                each_job['job_field'] = driver.find_element_by_xpath(str_fld).text
            except:
                str_fld = '//article[@data-job-id="'+id_str+'"]/div[1]/span[3]/span/span/strong/span/a'
                each_job['job_field'] = driver.find_element_by_xpath(str_fld).text

            try:
                str_fld1 = '//article[@data-job-id="'+id_str+'"]/div[1]/span[4]/span/span/span/a'
                each_job['job_subfield'] = driver.find_element_by_xpath(str_fld1).text
            except:
                str_fld1 = '//article[@data-job-id="'+id_str+'"]/div[1]/span[3]/span/span/span/a'
                each_job['job_subfield'] = driver.find_element_by_xpath(str_fld1).text

            try:
                str_des = '//article[@data-job-id="'+id_str+'"]/span[6]/span'
                each_job['job_des'] = driver.find_element_by_xpath(str_des).text
            except:
                each_job['job_des'] = 'no info'

            try:
                str_url = '//article[@data-job-id="'+id_str+'"]/span[2]/span/h1/a'
                url_job = driver.find_element_by_xpath(str_url).get_attribute('href')
                each_job['job_url'] = url_job
                inter_web = web_scrape_internal_web(url_job)
                each_job['job_date'] = inter_web[0]
                each_job['job_full_desc'] = inter_web[1]
            except:
                each_job['job_url'] = 'no info'
                each_job['job_date'] = 'no info'
                each_job['job_full_desc'] = 'no info'

            data_jobs.append(each_job)

        driver.close()
    return data_jobs

In [5]:
# Web scrape full job description web site
def web_scrape_internal_web(url_in):
    #print(url_in)
    driver_in = webdriver.Chrome(executable_path="./chromedriver/chromedriver",chrome_options=chrome_options)
    driver_in.get(url_in)
    job_desc = driver_in.find_element_by_xpath('//div[@data-automation="jobDescription"]').text

    #url_2 = driver.find_element_by_xpath('//div[@data-automation="jobDescription"]/span/span').text
    job_date = driver_in.find_element_by_xpath('//*[@id="app"]/div/div/div[4]/div/div[2]/div/div/div[2]/div/div[2]/div/aside/span/div[2]/div/section/dl/dd[1]/span/span').text

    list_ret = list((job_date, job_desc))
    driver_in.quit()
    #print(job_desc)
    #print(job_date)
    return list_ret


In [6]:
# Setting up parameters
search_keywords = ["data-science","data-engineer","data-analyst","machine-learning-engineer"]
locations = ["All-Melbourne-VIC", "All-Sydney-NSW"]
range_salary = ["30000-60000","70000-100000","120000-200000"]


In [10]:
# This code scrapes jobs from Seek's web site, store the information in Pandas, and 
# export the data set on a csv files. The csv files are generated per each salary range 
# and location.
for each_city in locations:
    for each_sal in range_salary:
        n_url = url_root+search_keywords[1]+'-jobs/in-'+each_city+'?salaryrange='+each_sal+'&salarytype=annual'
        print(n_url)
        driver_2 = webdriver.Chrome(executable_path="./chromedriver/chromedriver",chrome_options=chrome_options)

        driver_2.get(n_url)
        out_jobs = driver_2.find_elements_by_xpath('//*[@id="SearchSummary"]/span/h1/strong')[0].text
        jobs = out_jobs.replace(',', '')
        print(type(jobs))
        n_jobs = int(jobs)//20
        print(f'Number of jobs in this page: {jobs}')
        print(f'Number of web pages to process: {n_jobs}')

        list_urls_each = list_url_def(url_root,search_keywords[1],each_city,n_jobs,each_sal)

        data_jobs_each = web_scrape_func(list_urls_each,search_keywords[1],each_city,each_sal)

        df_jobs_each = pd.DataFrame(data_jobs_each)

        str_file = 'jobs-'+each_city+'-'+each_sal+'-DE.csv'
        df_jobs_each.to_csv(str_file,index=False)

        driver_2.close()

https://www.seek.com.au/data-engineer-jobs/in-All-Melbourne-VIC?salaryrange=30000-60000&salarytype=annual


<ipython-input-10-a824f292ef41>:5: DeprecationWarning: use options instead of chrome_options
  driver_2 = webdriver.Chrome(executable_path="./chromedriver/chromedriver",chrome_options=chrome_options)


<class 'str'>
Number of jobs in this page: 58
Number of web pages to process: 2


<ipython-input-4-162f1612d82f>:6: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path="./chromedriver/chromedriver",chrome_options=chrome_options)


https://www.seek.com.au/data-engineer-jobs/in-All-Melbourne-VIC?page=1&salaryrange=30000-60000&salarytype=annual


<ipython-input-5-d1af21a69f93>:3: DeprecationWarning: use options instead of chrome_options
  driver_in = webdriver.Chrome(executable_path="./chromedriver/chromedriver",chrome_options=chrome_options)


https://www.seek.com.au/data-engineer-jobs/in-All-Melbourne-VIC?page=2&salaryrange=30000-60000&salarytype=annual
https://www.seek.com.au/data-engineer-jobs/in-All-Melbourne-VIC?salaryrange=70000-100000&salarytype=annual
<class 'str'>
Number of jobs in this page: 228
Number of web pages to process: 11
https://www.seek.com.au/data-engineer-jobs/in-All-Melbourne-VIC?page=1&salaryrange=70000-100000&salarytype=annual
https://www.seek.com.au/data-engineer-jobs/in-All-Melbourne-VIC?page=2&salaryrange=70000-100000&salarytype=annual
https://www.seek.com.au/data-engineer-jobs/in-All-Melbourne-VIC?page=3&salaryrange=70000-100000&salarytype=annual
https://www.seek.com.au/data-engineer-jobs/in-All-Melbourne-VIC?page=4&salaryrange=70000-100000&salarytype=annual
https://www.seek.com.au/data-engineer-jobs/in-All-Melbourne-VIC?page=5&salaryrange=70000-100000&salarytype=annual
https://www.seek.com.au/data-engineer-jobs/in-All-Melbourne-VIC?page=6&salaryrange=70000-100000&salarytype=annual
https://www.se

In [11]:
# After creating different csv files, this code uploads all the csv files created
# earlier to generate a single data set.
df1 = pd.read_csv('jobs-All-Melbourne-VIC-30000-60000.csv')
df2 = pd.read_csv('jobs-All-Melbourne-VIC-70000-100000.csv')
df3 = pd.read_csv('jobs-All-Melbourne-VIC-120000-200000.csv')
df4 = pd.read_csv('jobs-All-Sydney-NSW-30000-60000.csv')
df5 = pd.read_csv('jobs-All-Sydney-NSW-70000-100000.csv')
df6 = pd.read_csv('jobs-All-Sydney-NSW-120000-200000.csv')

df7 = pd.read_csv('jobs-All-Melbourne-VIC-30000-60000-DS.csv')
df8 = pd.read_csv('jobs-All-Melbourne-VIC-70000-100000-DS.csv')
df9 = pd.read_csv('jobs-All-Melbourne-VIC-120000-200000-DS.csv')
df10 = pd.read_csv('jobs-All-Sydney-NSW-30000-60000-DS.csv')
df11 = pd.read_csv('jobs-All-Sydney-NSW-70000-100000-DS.csv')
df12 = pd.read_csv('jobs-All-Sydney-NSW-120000-200000-DS.csv')

df13 = pd.read_csv('jobs-All-Melbourne-VIC-30000-60000-DE.csv')
df14 = pd.read_csv('jobs-All-Melbourne-VIC-70000-100000-DE.csv')
df15 = pd.read_csv('jobs-All-Melbourne-VIC-120000-200000-DE.csv')
df16 = pd.read_csv('jobs-All-Sydney-NSW-30000-60000-DE.csv')
df17 = pd.read_csv('jobs-All-Sydney-NSW-70000-100000-DE.csv')
df18 = pd.read_csv('jobs-All-Sydney-NSW-120000-200000-DE.csv')

In [13]:
print(df1.shape, df2.shape, df3.shape, df4.shape, df5.shape, df6.shape)

(280, 16) (900, 16) (680, 16) (480, 16) (1500, 16) (1600, 16)


In [14]:
print(df7.shape, df8.shape, df9.shape, df10.shape, df11.shape, df12.shape)

(40, 16) (160, 16) (160, 16) (60, 16) (260, 16) (300, 16)


In [15]:
print(df13.shape, df14.shape, df15.shape, df16.shape, df17.shape, df18.shape)

(40, 16) (220, 16) (260, 16) (60, 16) (340, 16) (600, 16)


In [16]:
frames = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15,df16,df17,df18]

In [17]:
result_df = pd.concat(frames).reset_index()
result_df.drop(columns='index', inplace=True)
print(result_df.shape)
result_df.head(1)

(7940, 16)


,s_keyword,s_loc,range_sal,job_title,job_id,city,job_agency_url,job_agency,job_type,job_salary,job_field,job_subfield,job_des,job_url,job_date,job_full_desc
0,data-analyst,All-Melbourne-VIC,30000-60000,Excel Data Analyst,50354487,Melbourne,https://www.seek.com.au/Natural-Selection-Grou...,Natural Selection Group,This is a Contract/Temp job,no info,Information & Communication Technology,Other,Working with excel on customer quoting initiat...,https://www.seek.com.au/job/50354487?type=stan...,6 Aug 2020,Our Client has a new opportunity for a Excel D...


In [19]:
result_df.drop_duplicates(subset='job_id',keep='first',inplace=True, ignore_index=True)

In [20]:
result_df.shape

(5898, 16)

In [21]:
result_df.head()

,s_keyword,s_loc,range_sal,job_title,job_id,city,job_agency_url,job_agency,job_type,job_salary,job_field,job_subfield,job_des,job_url,job_date,job_full_desc
0,data-analyst,All-Melbourne-VIC,30000-60000,Excel Data Analyst,50354487,Melbourne,https://www.seek.com.au/Natural-Selection-Grou...,Natural Selection Group,This is a Contract/Temp job,no info,Information & Communication Technology,Other,Working with excel on customer quoting initiat...,https://www.seek.com.au/job/50354487?type=stan...,6 Aug 2020,Our Client has a new opportunity for a Excel D...
1,data-analyst,All-Melbourne-VIC,30000-60000,Product Analytics Specialist,50392356,Melbourne,https://www.seek.com.au/Openpay-jobs,Openpay Pty Ltd,This is a Full Time job,no info,Information & Communication Technology,Product Management & Development,Openpay (ASX: OPY) is an omni-channel payment ...,https://www.seek.com.au/job/50392356?type=stan...,13 Aug 2020,Product Analytics Specialist\nOpenpay (ASX: OP...
2,data-analyst,All-Melbourne-VIC,30000-60000,Graduate Database Developer/Analyst (permanent...,50374020,Melbourne,https://www.seek.com.au/jobs?advertiserid=3092...,MIEPOL,This is a Full Time job,no info,Information & Communication Technology,Developers/Programmers,We are seeking a Graduate Database Developer/A...,https://www.seek.com.au/job/50374020?type=stan...,11 Aug 2020,We are looking for a Graduate Database Develop...
3,data-analyst,All-Melbourne-VIC,30000-60000,Graduate Analyst Position - Melbourne,50372185,Melbourne,https://www.seek.com.au/Proforce-jobs,Proforce,This is a Full Time job,"$50,000 per annum",Sales,Analysis & Reporting,Graduate inventory and sales analyst position ...,https://www.seek.com.au/job/50372185?type=stan...,10 Aug 2020,The Business\n\nFounded in 2014 this Australia...
4,data-analyst,All-Melbourne-VIC,30000-60000,Business Analyst Junior or Emerging,50384667,Melbourne,https://www.seek.com.au/Interface-Recruitment-...,Interface Recruitment,This is a Full Time job,$60 - $65 plus super great ground floor role,Information & Communication Technology,Business/Systems Analysts,Want to join a medium sized Software House wit...,https://www.seek.com.au/job/50384667?type=stan...,12 Aug 2020,Business Analyst Junior or Emerging\nInterface...


In [22]:
result_df.nunique()

s_keyword            3
s_loc                2
range_sal            3
job_title         4455
job_id            5898
city                 2
job_agency_url    2228
job_agency        2295
job_type             4
job_salary        1409
job_field           29
job_subfield       258
job_des           5288
job_url           5898
job_date            70
job_full_desc     5529
dtype: int64

In [23]:
# Save scrapped information on a CSV for cleaning
result_df.to_csv('jobs_project4v2.csv',index=False)